# Stock Prediction with Support Vector Regressor

## Import Libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [3]:
import datetime
from pandas_datareader import data, wb

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [5]:
import warnings
warnings.filterwarnings("ignore") 

## Collecting Data

In [6]:
start_date = datetime.datetime(2010, 1, 1)
end_date = datetime.datetime.today()

#### Create DataFrame

In [7]:
SBI = data.DataReader("SBIN.NS", data_source='yahoo', start=start_date, end=end_date)
HDFC = data.DataReader("HDFCBANK.NS", data_source='yahoo', start=start_date, end=end_date)
AXIS = data.DataReader("AXISBANK.NS", data_source='yahoo', start=start_date, end=end_date)
BOB = data.DataReader("BANKBARODA.NS", data_source='yahoo', start=start_date, end=end_date)
PNB = data.DataReader("PNB.NS", data_source='yahoo', start=start_date, end=end_date)
ICICI = data.DataReader("ICICIBANK.NS", data_source='yahoo', start=start_date, end=end_date)
YESBANK = data.DataReader("YESBANK.NS", data_source='yahoo', start=start_date, end=end_date)
KOTAK = data.DataReader("KOTAKBANK.NS", data_source='yahoo', start=start_date, end=end_date)

## Getting Data Ready

In [8]:
# Get the Adj Close Price
sbi_df = SBI[["Adj Close"]]
hdfc_df = HDFC[["Adj Close"]]
axis_df = AXIS[["Adj Close"]]
bob_df = BOB[["Adj Close"]]
pnb_df = PNB[["Adj Close"]]
icici_df = ICICI[["Adj Close"]]
yesbank_df = YESBANK[["Adj Close"]]
kotak_df = KOTAK[["Adj Close"]]

In [9]:
# A variable for predicting 'n' days out into the future 
forecast_out = 30

# Create another column (the target or dependent variable) shifted 'n' units up
sbi_df['Prediction'] = sbi_df[['Adj Close']].shift(-forecast_out)
hdfc_df['Prediction'] = hdfc_df[['Adj Close']].shift(-forecast_out)
axis_df['Prediction'] = axis_df[['Adj Close']].shift(-forecast_out)
bob_df['Prediction'] = bob_df[['Adj Close']].shift(-forecast_out)
pnb_df['Prediction'] = pnb_df[['Adj Close']].shift(-forecast_out)
icici_df['Prediction'] = icici_df[['Adj Close']].shift(-forecast_out)
yesbank_df['Prediction'] = yesbank_df[['Adj Close']].shift(-forecast_out)
kotak_df['Prediction'] = kotak_df[['Adj Close']].shift(-forecast_out)

In [10]:
# List of the ticker symbols (as strings) in alphabetical order
tickers = 'SBI HDFCBANK AXISBANK BANKBARODA PNB ICICIBANK YESBANK KOTAKBANK'.split()
tickers.sort()

In [11]:
# Create a Single DataFrame by Concatenating all the above DataFrames
bank_stocks = pd.concat([axis_df, bob_df, hdfc_df, icici_df, kotak_df, pnb_df, sbi_df, yesbank_df], axis=1, keys=tickers)
bank_stocks = bank_stocks[:-30]
bank_stocks.head()

AXISBANK             BANKBARODA               HDFCBANK  \
             Adj Close  Prediction  Adj Close Prediction   Adj Close   
Date                                                                   
2010-01-04  140.712021  151.377823  51.206989  57.293846  134.245209   
2010-01-05  143.647934  155.802994  52.686508  58.495327  134.363205   
2010-01-06  141.236801  155.235626  53.009216  56.608700  134.453751   
2010-01-07  142.236725  157.072403  53.326965  56.161865  134.803970   
2010-01-08  144.094727  155.753357  53.540455  56.971127  134.981079   

                        ICICIBANK              KOTAKBANK              \
            Prediction  Adj Close Prediction   Adj Close  Prediction   
Date                                                                   
2010-01-04  130.329666  85.319977  81.561913  202.426041  186.522232   
2010-01-05  132.517624  86.130264  81.474457  204.430389  184.850494   
2010-01-06  133.316467  86.789574  80.844452  206.915863  179.573486   
2010-01-07  133.528961  85.970222  80.655098  209.240005  176.166992   
2010-01-08  133.875275  84.762535  82.221260  207.948059  173.558487   

                  PNB                   SBI               YESBANK             
            Adj Close Prediction  Adj Close Prediction  Adj Close Prediction  
Date                                                                          
2010-01-04  84.943443  86.630409  95.066895  85.587837  34.458496  31.044094  
2010-01-05  86.255913  87.150787  95.102180  84.955849  34.965523  30.293184  
2010-01-06  85.847481  86.114899  95.672699  83.333183  34.471333  29.439579  
2010-01-07  84.668106  85.934959  95.137444  83.816475  33.887291  30.107061  
2010-01-08  84.792007  85.910637  94.853218  83.621841  33.726841  30.312439

# Creating and Evaluating the Model

In [26]:
for i in range(len(tickers)):
    
    bank = bank_stocks[tickers[i]]
    
    # X and y
    X = np.array(bank.drop(['Prediction'], axis=1))
    y = np.array(bank["Prediction"])
    
    # Training and Testing Data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    # Instantiate Support Vector Regressor
    svr = SVR()
    svr.fit(X_train, y_train)
    
    # Evaluate Support Vector Regressor
    svr_confidence = svr.score(X_test, y_test)
    print(tickers[i])
    print("Support Vector Machine Confidence: {:.3f}%".format(svr_confidence*100))
    print()

AXISBANK
Support Vector Machine Confidence: 3.829%

BANKBARODA
Support Vector Machine Confidence: 45.499%

HDFCBANK
Support Vector Machine Confidence: -6.688%

ICICIBANK
Support Vector Machine Confidence: 13.853%

KOTAKBANK
Support Vector Machine Confidence: -2.714%

PNB
Support Vector Machine Confidence: 33.668%

SBI
Support Vector Machine Confidence: 31.019%

YESBANK
Support Vector Machine Confidence: -0.345%



# Grid Search

In [25]:
from sklearn.model_selection import GridSearchCV

# Different parameters for the grid search
param_grid = {"C":[0.1, 1, 10, 100, 1000], "gamma":[1, 0.1, 0.01, 0.001, 0.0001]}

# Initiate GridSearchCV
grid = GridSearchCV(estimator=SVR(), param_grid=param_grid, verbose=3)

In [29]:
output_result = list()

for i in range(len(tickers)):
    
    bank = bank_stocks[tickers[i]]
    
    # X and y
    X = np.array(bank.drop(['Prediction'], axis=1))
    y = np.array(bank["Prediction"])
    
    # Training and Testing Data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    # Initiate GridSearchCV
    grid = GridSearchCV(estimator=SVR(), param_grid=param_grid, verbose=3)
    
    # Fit the grid
    grid.fit(X_train, y_train)
    
    # Evaluate Support Vector Regressor
    grid_confidence = grid.score(X_test, y_test)
    
    # Results
    output_result.append((grid_confidence, grid.best_params_))

Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] C=0.1, gamma=1 ..................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ..................... C=0.1, gamma=1, score=-0.025, total=   0.3s
[CV] C=0.1, gamma=1 ..................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] ..................... C=0.1, gamma=1, score=-0.005, total=   0.3s
[CV] C=0.1, gamma=1 ..................................................


...

[CV] ..................... C=0.1, gamma=1, score=-0.271, total=   0.3s
[CV] C=0.1, gamma=1 ..................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s


[CV] ..................... C=0.1, gamma=1, score=-0.187, total=   0.3s
[CV] C=0.1, gamma=0.1 ................................................
[CV] ................... C=0.1, gamma=0.1, score=-0.101, total=   0.3s
[CV] C=0.1, gamma=0.1 ................................................
[CV] ................... C=0.1, gamma=0.1, score=-0.233, total=   0.3s
[CV] C=0.1, gamma=0.1 ................................................
[CV] ................... C=0.1, gamma=0.1, score=-0.151, total=   0.3s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] .................. C=0.1, gamma=0.01, score=-0.009, total=   0.2s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] .................. C=0.1, gamma=0.01, score=-0.102, total=   0.2s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] .................. C=0.1, gamma=0.01, score=-0.038, total=   0.2s
[CV] C=0.1, gamma=0.001 ..............................................
[CV] .

[CV] ..................... C=1000, gamma=1, score=0.879, total=   1.5s
[CV] C=1000, gamma=1 .................................................
[CV] ..................... C=1000, gamma=1, score=0.887, total=   1.6s
[CV] C=1000, gamma=1 .................................................
[CV] ..................... C=1000, gamma=1, score=0.890, total=   1.3s
[CV] C=1000, gamma=0.1 ...............................................
[CV] ................... C=1000, gamma=0.1, score=0.927, total=   1.0s
[CV] C=1000, gamma=0.1 ...............................................
[CV] ................... C=1000, gamma=0.1, score=0.917, total=   0.9s
[CV] C=1000, gamma=0.1 ...............................................
[CV] ................... C=1000, gamma=0.1, score=0.941, total=   0.9s
[CV] C=1000, gamma=0.01 ..............................................
[CV] .................. C=1000, gamma=0.01, score=0.942, total=   0.6s
[CV] C=1000, gamma=0.01 ..............................................
[CV] .

[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:   25.2s finished


# Results After Grid Search

In [36]:
for i in range(len(tickers)):   
    print(tickers[i])
    print("Support Vector Machine Confidence: {:.3f}%".format(output_result[i][0]*100))
    print("At Grid Best Parameters: ", output_result[i][1])
    print()

AXISBANK
Support Vector Machine Confidence: 91.628%
At Grid Best Parameters:  {'C': 100, 'gamma': 0.001}

BANKBARODA
Support Vector Machine Confidence: 85.070%
At Grid Best Parameters:  {'C': 1000, 'gamma': 0.001}

HDFCBANK
Support Vector Machine Confidence: 97.200%
At Grid Best Parameters:  {'C': 1000, 'gamma': 0.0001}

ICICIBANK
Support Vector Machine Confidence: 90.418%
At Grid Best Parameters:  {'C': 1000, 'gamma': 0.001}

KOTAKBANK
Support Vector Machine Confidence: 96.128%
At Grid Best Parameters:  {'C': 1000, 'gamma': 0.0001}

PNB
Support Vector Machine Confidence: 79.185%
At Grid Best Parameters:  {'C': 100, 'gamma': 0.001}

SBI
Support Vector Machine Confidence: 87.602%
At Grid Best Parameters:  {'C': 10, 'gamma': 0.001}

YESBANK
Support Vector Machine Confidence: 94.333%
At Grid Best Parameters:  {'C': 10, 'gamma': 0.001}

